In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [21]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [22]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [23]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [24]:
#handling dates in days
users.created_date=pd.to_datetime(users.created_date)
#users['today']=np.datetime64('2019-06-01')
users['account_duration'] = (np.datetime64('2019-06-01') - users['created_date']).dt.days

In [25]:
#users['month']=pd.DatetimeIndex(users['created_date']).month
#users['year']=pd.DatetimeIndex(users['created_date']).year
#users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [26]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [27]:
#tempo table me hmeromhnies twn transaction 
#Oi hmeromhnies ypologizontai se meres apo thn 1/6
tempo=trans[['user_id','created_date']]
tempo.sort_values(['user_id','created_date'],inplace=True)
tempo['created_date']=pd.to_datetime(tempo['created_date'])
tempo['days_from_transaction']=(np.datetime64('2019-06-01') - tempo['created_date']).dt.days
tempo.drop(columns=['created_date'], inplace=True)


c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [28]:
unique_days_of_trans.head(5)

,unique_days_of_transactions
user_id,
user_0,160
user_1,69
user_10,59
user_100,25
user_1000,15


In [29]:
#uni table exei tis unique dates poy exei kanei ka8e user transaction
uni = tempo.drop_duplicates(["user_id","days_from_transaction"])


In [30]:
#ypologizontai o mean tvn diaforwn kathe transaction, kai h max diafora transaction gia kathe user
mean_period_within_trans=uni.groupby(['user_id']).agg(mean_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).mean()))
max_period_within_trans=uni.groupby(['user_id']).agg(max_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).max()))
       

In [31]:
users=pd.merge(users , mean_period_within_trans, how='left', left_on='user_id', right_index=True)

In [32]:
users=pd.merge(users , max_period_within_trans, how='left', left_on='user_id', right_index=True)

In [33]:
#ypologizetai poses unique dates exei kanei transactions kathe user
unique_days_of_trans=tempo.groupby(['user_id']).agg(unique_days_of_transactions=('days_from_transaction', 'nunique'))

In [34]:
users=pd.merge(users , unique_days_of_trans, how='left', left_on='user_id', right_index=True)

In [35]:
#ypolofizetai to std, var twn dates of transactions gia ka8e user
std_var_trans=tempo.groupby('user_id').agg(std_of_transactions=('days_from_transaction','std'),var_of_transactions=('days_from_transaction','var'))


In [36]:
users=pd.merge(users , std_var_trans, how='left', left_on='user_id', right_index=True)

In [37]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [38]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [39]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [40]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [41]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

In [42]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

In [43]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [44]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [45]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [46]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [47]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [48]:
#handling dates in days
lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
#lasttrans['today']=np.datetime64('2019-06-01')
lasttrans['last_transaction'] = (np.datetime64('2019-06-01') - lasttrans['created_date']).dt.days

In [49]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [50]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [51]:
trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
firsttrans = trans_sorted_asc.drop_duplicates('user_id').sort_index()
firsttrans=firsttrans[['user_id','created_date']]

In [52]:
#handling dates in days
firsttrans['created_date']=pd.to_datetime(firsttrans['created_date'])
#firsttrans['today']=np.datetime64('2019-06-01')
firsttrans['first_transaction'] = (np.datetime64('2019-06-01')-firsttrans['created_date']).dt.days

In [53]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [54]:
users=pd.merge(users , firsttrans, how='left', left_on='user_id', right_on='user_id')

In [55]:
users.sample(4)

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction
12034,user_12034,1977,HU,Budapest,2018-04-19 18:49:27.079643,0,STANDARD,NaN,NaN,5,...,1.0,0.0,0.0,0.0,4.0,1.0,2019-05-12 07:32:33.284643,19.0,2018-04-19 18:56:08.235643,407.0
8497,user_8497,1986,GB,"Putney, London",2018-05-15 20:26:06.328943,0,STANDARD,NaN,NaN,0,...,0.0,0.0,0.0,0.0,8.0,0.0,2019-03-25 23:24:06.400943,67.0,2018-05-16 00:27:47.895943,380.0
5080,user_5080,1982,CZ,Nenačovice,2018-05-27 05:41:42.928838,1,STANDARD,1.0,1.0,12,...,0.0,0.0,0.0,0.0,5.0,0.0,2019-04-08 09:23:21.446838,53.0,2018-08-04 10:09:00.265838,300.0
50,user_50,1987,BE,Soumagne,2018-03-05 17:17:40.162006,0,STANDARD,NaN,NaN,1,...,0.0,1.0,0.0,0.0,1.0,0.0,2018-12-20 12:30:12.970006,162.0,2018-03-09 14:16:13.212006,448.0


In [56]:
users['engagement_duration']=users['first_transaction'] - users['last_transaction'] + 1

In [57]:
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date_x',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brands', 'age',
       'account_duration', 'mean_period_within_transactions',
       'max_period_within_transactions', 'unique_days_of_transactions',
       'std_of_transactions', 'var_of_transactions',
       'total_amount_of_transactions', 'amount_of_transactions_out',
       'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out',
       'amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in',
       'sum_usd_in', 'count_type_atm', 'count_type_card_payment',
       'count_type_exchange', 'count_type_fee', 'count_type_tax',
       'count_type_transfer_out', 'count_type_card_refund',
       'count_type_cashback', 'count_type_refund', 'count_type_topup',
       'count_type_transfer_in'

In [58]:
users['days_passed_for_first_transaction']= users['account_duration']- users['first_transaction'] 

In [59]:
users.sample()

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction,engagement_duration,days_passed_for_first_transaction
5571,user_5571,1995,GB,Coatbridge,2018-12-17 22:53:40.682116,0,STANDARD,NaN,NaN,0,...,0.0,0.0,2.0,0.0,2019-01-10 21:50:45.603116,141.0,2018-12-17 22:57:28.861116,165.0,25.0,0.0


In [60]:
users['transactions_frequency']=users['total_amount_of_transactions'] / users['engagement_duration']

#Notifications
________________________________________________________


In [61]:
notif_sent=notif[notif['status']=='SENT']


In [62]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=[notif_sent['channel'],notif_sent['reason']])
amount_notif_sent.columns=['_'.join(col).lower() for col in amount_notif_sent.columns.values]

In [63]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [64]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [65]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [66]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [67]:
#users[users['user_id']=='user_12544']

In [68]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [69]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [70]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [71]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [72]:
#dropping columns we don't need
data=users.drop(columns=['user_id','birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','num_referrals','num_successful_referrals','created_date_y','created_date'])


#'year_x','month_x','month_y','year_y','month','year'

In [73]:
data.columns

Index(['country', 'user_settings_crypto_unlocked', 'plan', 'num_contacts',
       'brands', 'age', 'account_duration', 'mean_period_within_transactions',
       'max_period_within_transactions', 'unique_days_of_transactions',
       'std_of_transactions', 'var_of_transactions',
       'total_amount_of_transactions', 'amount_of_transactions_out',
       'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out',
       'amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in',
       'sum_usd_in', 'count_type_atm', 'count_type_card_payment',
       'count_type_exchange', 'count_type_fee', 'count_type_tax',
       'count_type_transfer_out', 'count_type_card_refund',
       'count_type_cashback', 'count_type_refund', 'count_type_topup',
       'count_type_transfer_in', 'last_transaction', 'first_transaction',
       'engagement_duration', 'days_passed_for_first_transaction',
       'transactions_frequency', 'email_engagement_split_bill_restaurant',
       'email_invest_in_go

In [74]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [75]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [76]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [77]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [78]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
datatest.to_csv('latest_for_testing.csv',index=False, sep=';')
data.to_csv('latest_from_users.csv',index=False, sep=';')
users.to_csv('latest_users_feat.csv',index=False, sep=';')

In [79]:
datatest[datatest['transactions_frequency']==np.inf]

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple


In [81]:
datatest.head()

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple
0,0,0,3,27,263,1.522013,18.0,160.0,64.059353,4103.600767,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1
1,1,0,75,34,505,6.676471,79.0,69.0,104.200464,10857.736728,...,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0,1
2,0,0,71,23,183,2.238095,24.0,64.0,25.899183,670.767664,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0,1
3,0,0,219,38,468,5.544304,104.0,80.0,104.476260,10915.289003,...,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0,1
4,0,0,0,27,508,9.484848,71.0,34.0,66.093345,4368.330204,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0,1


In [80]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values